In [1]:
import numpy as np
qubit_0 =  np.array([1,0]).reshape(2,1)
qubit_1 =  np.array([0,1]).reshape(2,1)
qubit_00= np.kron(qubit_0,qubit_0)
qubit_11 = np.kron(qubit_1,qubit_1)
qubit_000 = np.kron(np.kron(qubit_0,qubit_0),qubit_0)
qubit_111 = np.kron(np.kron(qubit_1,qubit_1),qubit_1)
identity= np.array([1,0,0,1]).reshape(2,2)
p0 =  np.array([1,0,0,0]).reshape(2,2)
p1=  np.array([0,0,0,1]).reshape(2,2)
X = np.array([0,1,1,0]).reshape(2,2)
hadamard = (1/(np.sqrt(2))) * np.array([1,1,1,-1]).reshape(2,2)
cnot = np.kron(p0,identity) + ( np.kron(p1,X))






In [2]:
s1= np.matmul(hadamard,qubit_0)
s2=np.kron(s1,qubit_0)
s3=np.matmul(cnot,s2)
s4= np.kron(s3,qubit_0)
ghz = np.vstack((s4[:4],np.matmul(cnot,s4[4:])))
ghz

array([[0.70710678],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.70710678]])